In [52]:
def generate_5param_STC(datasheet_values):
    """
    
    This function will return the 5 parameters of a single diode model of the solar cell given datasheet values
    
    Input  : Datasheet Parameters: Voc, Isc, Vm, Im, Ncell at STC
             
    Output : Returns the single diode parameter tuple (Rs, Rsh, Isat, Iph, eta)
    
    
    """
    Voc, Isc, Vm, Im, Ncell = datasheet_values
    
    Temp = 25
    kb = 1.38*10**(-23) #Boltzmann constant
    q = 1.6*10**(-19)
    Vt = kb*(Temp+273)/q
    
    #Initial Guess Calculation (eta, Rs, Rsh)
    etai = (2*Vm-Voc) / (Ncell*Vt*(np.log((Isc-Im)/Isc)+Im/(Isc-Im)))
    Rsi = Vm/Im - (2*Vm-Voc) / ((Isc - Im) * (np.log((Isc-Im)/Isc) + Im/(Isc-Im)))
    if (Rsi<0):
        Rsi = 0
        num = 1 + (1-Im/Isc)*(np.log(1-Im/Isc))/(1-Vm/Voc)
        den = Im/Vm + (Isc-Im)*np.log(1-Im/Isc)/(Voc-Vm)
        Rshi = num/den
    else :
        Rshi = np.sqrt(Rsi/((Isc/(etai*Ncell*Vt)*np.exp((Rsi*Isc - Voc)/(etai*Ncell*Vt)))))

    # Solving them to get these 3 params
    def para(X):
        eta, Rs, Rsh = X
        f = [Im - Isc + (Vm + Im*Rs - Isc*Rs)/Rsh + (Isc - (Voc - Isc)/Rsh)*np.exp((-Voc + Vm+Im*Rs)/(eta*Ncell*Vt)), Im + Vm*((-((-Voc+Isc*Rsh+Isc*Rs)/(eta*Ncell*Vt*Rsh))*(np.exp((-Voc + Vm+Im*Rs)/(eta*Ncell*Vt)))-(1/Rsh))/(1+((-Voc+Isc*Rsh+Isc*Rs)/(eta*Ncell*Vt*Rsh)*Rs*np.exp((-Voc+Vm+Im*Rs)/(eta*Ncell*Vt)))+Rs/Rsh)), 1/Rsh + ((-((-Voc+Isc*Rsh+Isc*Rs)/(eta*Ncell*Vt*Rsh))*(np.exp((-Voc + Isc*Rs)/(eta*Ncell*Vt)))-(1/Rsh))/(1+((-Voc+Isc*Rsh+Isc*Rs)/(eta*Ncell*Vt*Rsh)*Rs*np.exp((-Voc+Isc*Rs)/(eta*Ncell*Vt)))+Rs/Rsh))]
        return f

    param_sol = root(para,[etai,Rsi,Rshi])
    eta = param_sol.x[0]
    Rs = param_sol.x[1]
    Rsh = param_sol.x[2]
    if (Rs<0):
        Rs=0
    # Getting the remaining 2 parameters
    Isat = (Isc - (Voc - Isc*Rs)/Rsh)*(np.exp(-Voc/(eta*Ncell*Vt)));
    Iph = Isat * np.exp(Voc / (eta*Ncell*Vt)) + (Voc/Rsh);

    parameters = (Rs, Rsh, Isat, Iph, eta)
    
    return (parameters)
#print (generate_5param_STC((45.9, 9.25, 37.2, 8.76, 72)))

(0.3551735739069161, 20051.491203008736, 2.9778179325966427e-10, 9.250163845946686, 1.0266558791833786)


In [78]:
def generate_IV(diode_param, Temp, datapoints, Ncell, Voc=None):
    """
    This function will generate I-V data points given all the 5 diode parameters 
    
    Input  : 5 parameters: Rs, Rsh, Isat, Iph, eta
             Voc, Temperature(Celcius), N(Number of data points), Ncell
             
    Output : Returns a dictionary of I-V data (size = N)
    
    This function will spit out I-V data points. 
    
    ** This auto generates Voc also if not provided **
    **Temperature is just taken to calculate Vt**
    
    """
    kb = 1.38*10**(-23) #Boltzmann constant
    q = 1.6*10**(-19)
    Vt = kb*(Temp+273)/q
    
    Rs, Rsh, Isat, Iph, eta = diode_param
    func = lambda i: -i + Iph - Isat*(np.exp((v+i*Rs) / (eta*Ncell*Vt))-1) - (v+i*Rs)/Rsh
    
    if not Voc:
        calv = lambda voc: Iph - Isat*(np.exp((voc) / (eta*Ncell*Vt))-1) - (voc)/Rsh
        Vocr = root(calv,50)
        Voc = Vocr.x[0]
        print ('Auto generated Voc is :', Voc)
    V = np.linspace(0,Voc,datapoints)
    I = []
    for v in V:
        ior = root(func,0.001)
        io = ior.x[0]
        I.append(io)
    I = np.round(I, 2)
    V = np.round(V, 2)
    return dict(x=V, y=I)

In [79]:
from bokeh.models import Slider, CustomJS
from bokeh.models.widgets import Button, TextInput
from bokeh.layouts import row, column, gridplot
from bokeh.plotting import Figure, output_notebook, show, ColumnDataSource, output_file
import pandas as pd
from scipy.optimize import root
import numpy as np
from os.path import dirname, join

output_notebook()

def plot_iv(doc):
    
    diode_param = (0.3551735739069161, 20051.491203008736, 2.9778179325966427e-10, 9.250163845946686, 1.0266558791833786)
    source = ColumnDataSource(data=generate_IV(diode_param, 25, 100, 72))

    plot = Figure(y_range = (-1, 10), x_range = (0,60))
    plot.xaxis.axis_label = 'Voltage (V)'
    plot.yaxis.axis_label = 'Current (I)'
    plot.scatter('x', 'y', source=source, line_width=3, line_alpha=0.6)
    
    Rs_input = TextInput(value='0.35', title='Series Resistance')
    Rsh_input = TextInput(value='20051', title='Shunt Resistance')
    Isat_input = TextInput(value='2.97e-10', title='Isat')
    Iph_input = TextInput(value='9.25', title='Iph')
    eta_input = TextInput(value='1.02', title='Eta (Ideality Factor)')
    submit = Button(label='Submit', button_type='success')

    def get_inputs():
        return (float(Rs_input.value) ,float(Rsh_input.value) ,float(Isat_input.value) ,float(Iph_input.value) ,float(eta_input.value),)
    
    def update_plot(event):
        params = get_inputs()
        source.data = generate_IV(params, 25, 100, 72)

    submit.on_click(update_plot)
    layout = row(plot, column(Rs_input, Rsh_input, Isat_input, Iph_input, eta_input, submit),)
    doc.add_root(layout)

show(plot_iv)

Loading BokehJS ...

Auto generated Voc is : 45.90000000006135
Auto generated Voc is : 45.607358914699745
Auto generated Voc is : 44.86713185309501
